In [ ]:
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality

# Text example

An example of creating a language model and then transfering to a classifier.

In [ ]:
path = Path('/media/discoD/fastai/data/wiki/pt')
path

PosixPath('/media/discoD/fastai/data/wiki/pt')

In [ ]:
itos = pickle.load(open(path/'tmp'/'itos.pkl', 'rb'))
len(itos)

FileNotFoundError: [Errno 2] No such file or directory: '/media/discoD/fastai/data/wiki/pt/tmp/itos.pkl'

In [ ]:
path = Path('/media/discoD/Corpora/Anexos Trabalhistas/200kk')
#path = Path('/media/discoD/fastai/data/wiki/pt')
path

PosixPath('/media/discoD/Corpora/Anexos Trabalhistas/200kk')

In [ ]:
itos = pickle.load(open(path/'tmp'/'itos.pkl', 'rb'))
len(itos)

60002

In [ ]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(stoi)

60002

In [ ]:
tok_trn = np.load(path/'tmp'/'train_ids.npy')
tok_tst = np.load(path/'tmp'/'test_ids.npy')
tok_val = np.load(path/'tmp'/'valid_ids.npy')

In [ ]:
freq = Counter(p for o in tok_trn for p in o)

In [ ]:
[(itos[term], count) for (term, count) in freq.most_common(25)]

[('xxup', 22962801),
 (',', 16808634),
 ('de', 10960125),
 ('.', 8879642),
 ('a', 7597534),
 ('\n', 7336655),
 ('do', 6920493),
 ('o', 5706833),
 ('-', 5387489),
 ('\n ', 4825104),
 ('da', 4761271),
 (')', 4482108),
 ('que', 3660299),
 ('e', 3565020),
 (':', 3070489),
 ('xxunk', 2622662),
 ('trabalho', 2586286),
 ('/', 2522491),
 ('se', 2392844),
 ('em', 2374612),
 ('(', 1666518),
 ('os', 1625193),
 ('não', 1587663),
 ('no', 1577188),
 ('para', 1405769)]

In [ ]:
data_lm = TextLMDataBunch.from_id_files(path=path, train='train', valid='valid', test='test', max_vocab=60000,
                                        min_freq=2, n_labels=0, chunksize=24000)

In [ ]:
#data_lm = TextLMDataBunch.from_csv(path=path, tokenizer=Tokenizer(lang='pt'), train='train', valid='val', 
#                                   max_vocab=60000, min_freq=2, n_labels=0, chunksize=24000)

Open and view the independent and dependent variables:

Create a `DataBunch` for each of the language model and the classifier:

In [ ]:
#data_lm = TextLMDataBunch.from_csv(path)
#data_clas = TextClasDataBunch.from_csv(path, vocab=data_lm.train_ds.vocab)

[fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download.

In [ ]:
datasets.download_wt103_model()

We'll fine-tune the language model:

In [ ]:
learn = RNNLearner.language_model(data_lm)

In [ ]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 00:18
epoch  train loss  valid loss  accuracy
0      4.931205    4.158325    0.244630  (00:09)
1      4.660501    4.097024    0.252083  (00:09)



Save our language model's encoder:

In [ ]:
learn.save_encoder('enc')

Fine tune it to create a classifier:

In [ ]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder('enc')
learn.fit(3, 1e-3)

Total time: 00:21
epoch  train loss  valid loss  accuracy
0      0.709290    0.672736    0.610000  (00:07)
1      0.685293    0.647017    0.630000  (00:06)
2      0.662346    0.609948    0.660000  (00:07)

